In [5]:
!pip install prophet
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import plotly.graph_objects as go
import plotly.io as pio

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'prophet'

In [2]:
renewable_p= pd.read_csv('modern-renewable-production.csv')
renewable_c = pd.read_csv('renewable-energy-consumption-share.csv')

In [3]:
renewable_p

,Entity,Code,Year,Electricity from wind - TWh,Electricity from hydro - TWh,Electricity from solar - TWh,Other renewables including bioenergy - TWh
0,ASEAN (Ember),NaN,2000,0.0,50.45,0.00,22.800000
1,ASEAN (Ember),NaN,2001,0.0,54.33,0.00,22.860000
2,ASEAN (Ember),NaN,2002,0.0,53.29,0.00,23.230000
3,ASEAN (Ember),NaN,2003,0.0,53.21,0.00,23.189999
4,ASEAN (Ember),NaN,2004,0.0,52.99,0.00,24.970000
...,...,...,...,...,...,...,...
9203,Zimbabwe,ZWE,2018,0.0,5.05,0.02,0.190000
9204,Zimbabwe,ZWE,2019,0.0,4.17,0.03,0.190000
9205,Zimbabwe,ZWE,2020,0.0,3.81,0.02,0.100000
9206,Zimbabwe,ZWE,2021,0.0,5.93,0.02,0.000000


In [4]:
renewable_p_AUS_JPN = renewable_p[(renewable_p['Entity'] == 'Australia') | (renewable_p['Entity'] == 'Japan')]

renewable_p_AUS_JPN = renewable_p_AUS_JPN[['Entity', 'Year', 'Electricity from hydro - TWh', 'Other renewables including bioenergy - TWh', 'Electricity from wind - TWh']]

renewable_p_AUS_JPN.rename(columns = {'Entity':'Country', 'Electricity from hydro - TWh':'Hydro Energy-TWh',
                                      'Other renewables including bioenergy - TWh':'Other Renewables-TWh',
                                      'Electricity from wind - TWh':'Wind Energy-TWh'}, inplace = True)

renewable_p_AUS_JPN.reset_index(drop = True, inplace = True)

renewable_p_AUS_JPN.head()

,Country,Year,Hydro Energy-TWh,Other Renewables-TWh,Wind Energy-TWh
0,Australia,1965,7.689756,0.296,NaN
1,Australia,1966,7.345043,0.284,NaN
2,Australia,1967,7.605787,0.290,NaN
3,Australia,1968,7.950501,0.282,NaN
4,Australia,1969,8.684121,0.281,NaN


In [5]:
renewable_c_AUS_JPN = renewable_c[(renewable_c['Entity'] == 'Australia') | (renewable_c['Entity'] == 'Japan')]

renewable_c_AUS_JPN = renewable_c_AUS_JPN.rename(columns={'Entity': 'Country', 'Renewables (% equivalent primary energy)': '% Renewables Consumption'})

renewable_c_AUS_JPN = renewable_c_AUS_JPN[['Country','Year','% Renewables Consumption']]

renewable_c_AUS_JPN.reset_index(drop=True, inplace=True)

renewable_c_AUS_JPN


,Country,Year,% Renewables Consumption
0,Australia,1965,5.562622
1,Australia,1966,5.071895
2,Australia,1967,4.949549
3,Australia,1968,4.865468
4,Australia,1969,5.086988
...,...,...,...
113,Japan,2019,9.284181
114,Japan,2020,10.928532
115,Japan,2021,11.268134
116,Japan,2022,11.606308


In [6]:
renewable_p_AUS_JPN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               118 non-null    object 
 1   Year                  118 non-null    int64  
 2   Hydro Energy-TWh      118 non-null    float64
 3   Other Renewables-TWh  118 non-null    float64
 4   Wind Energy-TWh       93 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 4.7+ KB


In [7]:
renewable_p_AUS_JPN.describe()

,Year,Hydro Energy-TWh,Other Renewables-TWh,Wind Energy-TWh
count,118.000000,118.000000,118.000000,93.000000
mean,1994.000000,47.398191,7.810806,3.713755
std,17.102007,33.756562,9.889257,6.702807
min,1965.000000,7.345043,0.000000,0.000000
25%,1979.250000,14.355833,0.457000,0.000000
50%,1994.000000,42.016000,2.801025,0.130000
75%,2008.750000,80.707250,15.133882,5.010000
max,2023.000000,94.416000,42.045246,33.130000


In [8]:
renewable_p_AUS_JPN.nunique()

Country                   2
Year                     59
Hydro Energy-TWh        118
Other Renewables-TWh    112
Wind Energy-TWh          59
dtype: int64

In [9]:
print(f'The shape of the dataset is {renewable_p_AUS_JPN.shape}\n')

print('There are', renewable_p_AUS_JPN.duplicated().sum(), 'duplicated rows in the dataframe.')

renewable_p_AUS_JPN[renewable_p_AUS_JPN.duplicated()].shape

The shape of the dataset is (118, 5)

There are 0 duplicated rows in the dataframe.


(0, 5)

In [10]:
renewable_p_AUS_JPN.isnull().sum()

Country                  0
Year                     0
Hydro Energy-TWh         0
Other Renewables-TWh     0
Wind Energy-TWh         25
dtype: int64

In [11]:
renewable_c_AUS_JPN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Country                   118 non-null    object 
 1   Year                      118 non-null    int64  
 2   % Renewables Consumption  118 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.9+ KB


In [12]:
renewable_c_AUS_JPN.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,118.0,1994.000000,17.102007,1965.000000,1979.250000,1994.000000,2008.75000,2023.000000
% Renewables Consumption,118.0,6.080202,2.270364,3.681326,4.693712,5.466783,6.43991,14.982515


In [13]:
renewable_c_AUS_JPN.nunique()

Country                       2
Year                         59
% Renewables Consumption    118
dtype: int64

In [14]:
renewable_c_AUS_JPN.isnull().sum()

Country                     0
Year                        0
% Renewables Consumption    0
dtype: int64

In [15]:
print(f'The shape of the dataset is {renewable_c_AUS_JPN.shape}\n')

print('There are', renewable_c_AUS_JPN.duplicated().sum(), 'duplicated rows in the dataframe.')

renewable_c_AUS_JPN[renewable_c_AUS_JPN.duplicated()].shape

The shape of the dataset is (118, 3)

There are 0 duplicated rows in the dataframe.


(0, 3)

# ---------------------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------------

#                                                                      * Analysis  & Predictions * 

# Forecasting the renewable energy resources (Hydro and Wind) production in Australia.

In [16]:
# Creating dataframe for Australia's energy resources
aus_df= renewable_p_AUS_JPN[renewable_p_AUS_JPN['Country'] == 'Australia'][['Year', 'Hydro Energy-TWh', 'Wind Energy-TWh']].copy()

# Creating a "ds" that has the years in the format of YYYY.
aus_df['ds'] = pd.to_datetime(aus_df['Year'], format = '%Y')

## Forecasting Hydero Energy resource.

In [17]:
#### Modeling  Hydro Energy ####

# Renamming the columns of the target variable and the date series columns to be used for prophet modell
df_hydro_aus = aus_df[['ds', 'Hydro Energy-TWh']].rename(columns = {'Hydro Energy-TWh': 'y'})

# Initialising the model 
modeling_hydro_aus = Prophet()

# Fitting and training the model
modeling_hydro_aus.fit(df_hydro_aus)

# Creating dataFrame with future years for the next 20 years
future_hydro_aus = modeling_hydro_aus.make_future_dataframe(periods = 20, freq = 'Y')

# Creating future years predictions for the hydro energy resource
forecast_hydro_aus = modeling_hydro_aus.predict(future_hydro_aus)

15:16:33 - cmdstanpy - INFO - Chain [1] start processing
15:16:33 - cmdstanpy - INFO - Chain [1] done processing


## Forecasting wind Energy resource.

In [18]:
#### Modeling wind Energy ####

# Renamming the columns of the target variable and the date series columns to be used for prophet modell
df_wind_aus = aus_df[['ds', 'Wind Energy-TWh']].rename(columns = {'Wind Energy-TWh': 'y'})

# Initialising the model
modeling_wind_aus = Prophet()

# Fitting and training the model
modeling_wind_aus.fit(df_wind_aus)

# Creating dataFrame with future years for the next 20 years
future_wind_aus = modeling_wind_aus.make_future_dataframe(periods = 20, freq = 'Y')

# Creating future years predictions dataframe for the hydro energy resource
forecast_wind_aus = modeling_wind_aus.predict(future_wind_aus)

# Removing negative predictions from wind energy forecasting
forecast_wind_aus['yhat'] = forecast_wind_aus['yhat'].apply(lambda x: max(0, x))

15:16:33 - cmdstanpy - INFO - Chain [1] start processing
15:16:34 - cmdstanpy - INFO - Chain [1] done processing


In [19]:
# Renaming hydro energy dataframe columns to display them on the plot 
forecast_hydro_aus.rename(columns = {'ds': 'Years', 'yhat':'Predicted Hydro Energy (TWh)'}, inplace = True)

# Renaming wind energy dataframe columns to display them on the plot 
forecast_wind_aus.rename(columns = {'ds':'Year', 'yhat':'Predicted Wind Energy (TWh)'}, inplace = True)

In [20]:
# Opening the plot in the browser 
pio.renderers.default = 'browser'

# Creating initial empty plotly figure 
fig = go.Figure()

# making hydro energy resource data plot as default
fig.add_trace(go.Scatter(x = forecast_hydro_aus['Years'], y = forecast_hydro_aus['Predicted Hydro Energy (TWh)'], mode = 'lines', name = 'Hydro Energy', line = dict(color = 'crimson')))


# Updating the layout for the plot to add tites for the plot and axies
fig.update_layout(title = 'Energy Production Forecast for Australia', xaxis_title='Years', yaxis_title=' Generated Energy (TWh)')


# Adding dropdown list for selecting Hydro or Wind Energy resource

fig.update_layout(
    
    updatemenus = [
        dict(buttons = list([
                
                dict(args = [{'y': [forecast_hydro_aus['Predicted Hydro Energy (TWh)']], 'name': ['Hydro Energy'],
                           'yaxis.range': [0, max(forecast_hydro_aus['Predicted Hydro Energy (TWh)']) + 5]}],
                    label = "Hydro Energy",
                    method = "update"),
            
                
                dict(args = [{'y': [forecast_wind_aus['Predicted Wind Energy (TWh)']], 'name': ['Wind Energy'],
                           'yaxis.range': [0, max(forecast_wind_aus['Predicted Wind Energy (TWh)']) + 5]}],  
                    label = "Wind Energy",
                    method = "update")]),
            
            direction = "down")])

# Show the plot
fig.show()
fig.write_html("Australia_Hydro_and_Wind_renewables.html")

# Forecasting the renewable energy resources (Hydro and Wind) production in Japan.

In [21]:
# Creating dataframe for Japan's energy resources
jpn_df = renewable_p_AUS_JPN[renewable_p_AUS_JPN['Country'] == 'Japan'][['Year', 'Hydro Energy-TWh', 'Wind Energy-TWh']].copy()

# Creating a "ds" that has the years in the format of YYYY.
jpn_df['ds'] = pd.to_datetime(jpn_df['Year'], format = '%Y')

## Forecasting Hydero Energy resource.

In [22]:
#### Modeling  Hydro Energy ####


# Renamming the columns of the target variable and the date series columns to be used for prophet modell
df_hydro_jpn = jpn_df[['ds', 'Hydro Energy-TWh']].rename(columns = {'Hydro Energy-TWh': 'y'})

# Initialising the model
modeling_hydro_jpn = Prophet()

# Fitting and training the model
modeling_hydro_jpn.fit(df_hydro_jpn)

# Creating dataFrame with future years for the next 20 years
future_hydro_jpn = modeling_hydro_jpn.make_future_dataframe(periods = 20, freq = 'Y')

# Creating future years predictions for the hydro energy resource
forecast_hydro_jpn = modeling_hydro_jpn.predict(future_hydro_jpn)

15:16:36 - cmdstanpy - INFO - Chain [1] start processing
15:16:37 - cmdstanpy - INFO - Chain [1] done processing


## Forecasting wind Energy resource.

In [23]:
#### Modeling wind Energy ####


# Renamming the columns of the target variable and the date series columns to be used for prophet model
df_wind_jpn = jpn_df[['ds', 'Wind Energy-TWh']].rename(columns={'Wind Energy-TWh': 'y'})

# Initialising the model
modeling_wind_jpn = Prophet()

# Fitting and training the model
modeling_wind_jpn.fit(df_wind_jpn)

# Creating dataFrame with future years for the next 20 years
future_wind_jpn = modeling_wind_jpn.make_future_dataframe(periods = 20, freq = 'Y')

# Creating future years predictions dataframe for the hydro energy resource
forecast_wind_jpn = modeling_wind_jpn.predict(future_wind_jpn)

# Removing negative predictions from wind energy forecasting
forecast_wind_jpn['yhat'] = forecast_wind_jpn['yhat'].apply(lambda x: max(0, x))

15:16:37 - cmdstanpy - INFO - Chain [1] start processing
15:16:38 - cmdstanpy - INFO - Chain [1] done processing


In [24]:
# Renaming hydro energy dataframe columns to display them on the plot 
forecast_hydro_jpn.rename(columns = {'ds':'Year', 'yhat':'Predicted Hydro Energy (TWh)'}, inplace = True)

# Renaming wind energy dataframe columns to display them on the plot 
forecast_wind_jpn.rename(columns = {'ds':'Year', 'yhat':'Predicted Wind Energy (TWh)'}, inplace = True)

In [25]:
# Opening the plot in the browser 
pio.renderers.default = 'browser'

# Creating initial empty plotly figure 
fig = go.Figure()

# making hydro energy resource data plot as default
fig.add_trace(go.Scatter(x = forecast_hydro_jpn['Year'], y = forecast_hydro_jpn['Predicted Hydro Energy (TWh)'], mode = 'lines', name = 'Hydro Energy', line = dict(color = 'crimson')))

# Updating the layout for the plot to add tites for the plot and axies
fig.update_layout(title = 'Energy Production Forecast for Japan', xaxis_title = 'Year', yaxis_title = 'Energy (TWh)')


# Adding dropdown list for selecting Hydro or Wind Energy resource
fig.update_layout(
    
    updatemenus = [
        dict(buttons = list([
                
                dict(args = [{'y': [forecast_hydro_jpn['Predicted Hydro Energy (TWh)']], 'name': ['Hydro Energy'],
                           'yaxis.range': [0, max(forecast_hydro_jpn['Predicted Hydro Energy (TWh)']) + 5]}],
                    label = "Hydro Energy",
                    method = "update"),
            
                
                dict(args = [{'y': [forecast_wind_jpn['Predicted Wind Energy (TWh)']], 'name': ['Wind Energy'],
                           'yaxis.range': [0, max(forecast_wind_aus['Predicted Wind Energy (TWh)']) + 5]}],  
                    label = "Wind Energy",
                    method = "update")]),
            
            direction = "down")])

# Show the plot
fig.show()
fig.write_html("Japan_Hydro_and_Wind_renewables.html")

# Forecasting the consumption of renewable energy (% equivalent primary energy) in bothe countries, Australia & Japan. 

## Forecasting of consumption of renewable energy in Australia

In [26]:
# Creating dataframe for Australia's consumption of renewable energy
df_aus = renewable_c_AUS_JPN[renewable_c_AUS_JPN['Country'] == 'Australia'][['Year', '% Renewables Consumption']].copy()

# Creating a "ds" that has the years in the format of YYYY.
df_aus['ds'] = pd.to_datetime(df_aus['Year'], format = '%Y')

# Renamming the column of the target variable and the date series columns to be used for prophet model
df_aus.rename(columns = {'% Renewables Consumption':'y'}, inplace = True)


# Building fitting the model
modeling_aus = Prophet()
modeling_aus.fit(df_aus)

future_aus = modeling_aus.make_future_dataframe(periods = 20, freq = 'Y')
forecast_aus = modeling_aus.predict(future_aus)

15:16:40 - cmdstanpy - INFO - Chain [1] start processing
15:16:40 - cmdstanpy - INFO - Chain [1] done processing


## Forecasting of consumption of renewable energy in Japan

In [27]:
# Creating dataframe for Australia's consumption of renewable energy
df_jpn = renewable_c_AUS_JPN[renewable_c_AUS_JPN['Country'] == 'Japan'][['Year', '% Renewables Consumption']].copy()

# Creating a "ds" that has the years in the format of YYYY.
df_jpn['ds'] = pd.to_datetime(df_jpn['Year'], format = '%Y')

# Renamming the column of the target variable and the date series columns to be used for prophet model
df_jpn.rename(columns = {'% Renewables Consumption': 'y'}, inplace = True)


# Building fitting the model
modeling_jpn = Prophet()
modeling_jpn.fit(df_jpn)

future_jpn = modeling_jpn.make_future_dataframe(periods = 20, freq = 'Y')
forecast_jpn = modeling_jpn.predict(future_jpn)

15:16:41 - cmdstanpy - INFO - Chain [1] start processing
15:16:41 - cmdstanpy - INFO - Chain [1] done processing


In [30]:
# Opening the plot in the browser 
pio.renderers.default = 'browser'


# Creating the plot to show both Australia and Japan
fig = go.Figure()

# Adding Australia forecast as a blue line
fig.add_trace(go.Scatter(x = forecast_aus['ds'], y = forecast_aus['yhat'], mode = 'lines', name = 'Australia', line = dict(color = 'blue')))

# Adding Japan forecast as a red line

fig.add_trace(go.Scatter(x = forecast_jpn['ds'], y = forecast_jpn['yhat'], mode = 'lines', name = 'Japan', line = dict(color = 'crimson')))


# Updating the layout for the plot

fig.update_layout(title = 'Renewable Energy Consumption Forecast: Australia vs Japan',
                  xaxis_title='Year', yaxis_title='% Renewables Consumption',
                  legend = dict(x = 0.1, y = 1.1, orientation = 'h'))

# Show the plot
fig.show()
fig.write_html("Australia_and_Japan_Consumption.html")